<a href="https://colab.research.google.com/github/ehsanh123/Flower-type-detection-/blob/main/Alice_in_wonderland_deeplearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from keras.layers import Dense, Activation
#Simple RNN architecture
from keras.layers import SimpleRNN
from keras.models import Sequential

import numpy as np


#load data

In [12]:
#corpus = open("https://raw.githubusercontent.com/elboudab/DEL_datasets/main/alice_in_wonderland.txt", 'rb')
import requests

# Fetch the content from the URL
url = "https://raw.githubusercontent.com/elboudab/DEL_datasets/main/alice_in_wonderland.txt"
response = requests.get(url)

# Decode the content as text
corpus = response.content.decode("ascii", "ignore")

#Create documents list

documents = []

#Iterate through the corpus

# for document in corpus:
#   document = document.strip().lower()
#   document = document.decode("ascii", "ignore")
for line in corpus.splitlines(): # Change here: Iterate over lines
  document = line.strip().lower()
  # if you find empty line skip to next iteration
  if len(document) == 0:
    continue
  #Add the new document to the documents
  documents.append(document)

#Close the corpus
#corpus.close()
#print(corpus)
documents = "".join(documents)

In [13]:
#get unique words
chars = set([c for c in documents])
#get number of words
nb_chars = len(chars)
#assign and index to each word and make dictionary
char2index = dict((c, 1) for i, c in enumerate(chars))
#assgin index to words
index2char = dict((i, c) for i, c in enumerate(chars))

In [14]:
#The input width
SEQLEN = 10
#The output width
STEP = 1
#input words
input_chars = []
#output words
label_chars = []
#Iterate through documents and get input and labels
for i in range(0, len(documents) - SEQLEN, STEP):
  input_chars.append(documents[i:i + SEQLEN])
  label_chars.append(documents[i + SEQLEN])

In [16]:
#One hot encode the characters.
#x = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
x = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=bool)
#y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=bool)
for i, input_char in enumerate(input_chars):
  for j, ch in enumerate(input_char):
    x[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

#model 1

In [17]:
#hyper partmates
HIDDEN_UNITS = 128
BATCH_SIZE = 128
N_ITERATIONS = 25
N_EPOCHS_PER_ITERATION = 1
N_PREDS_PER_EPOCH = 100

In [18]:
model = Sequential()
#Simple RNN with width and number of characters as input
model.add(SimpleRNN(HIDDEN_UNITS, return_sequences=False,
  input_shape=(SEQLEN, nb_chars),
  unroll=True))

# Dynamic output based on the characters length.
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [19]:
#Loop for 25 times until 24
for iteration in range(N_ITERATIONS):
  #For each iteration
  print("\n")
  print("Iteration #: %d" % (iteration))
  # Fit iteration with one epoch
  model.fit(x, y, batch_size=BATCH_SIZE, epochs=N_EPOCHS_PER_ITERATION)
  #pick random words and test the model
  test_idx= np.random.randint(len(input_chars))
  test_chars = input_chars[test_idx]
  print("Generating from seed: %s" % (test_chars))
  print(test_chars, end="")
  # Predict 100 characters.
  for i in range(N_PREDS_PER_EPOCH):
    Xtest = np.zeros((1, SEQLEN, nb_chars))
    for i, ch in enumerate(test_chars):
      Xtest[0, i, char2index[ch]] = 1
    pred = model.predict(Xtest, verbose=0)[0]
    ypred = index2char[np.argmax(pred)]
    print(ypred, end="") # move forward with test_chars + ypred
    test_chars = test_chars[1:] + ypred



Iteration #: 0
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - loss: 0.0507
Generating from seed: ceeded in 
ceeded in ````````````````````````````````````````````````````````````````````````````````````````````````````

Iteration #: 1
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 2.2033e-06
Generating from seed: ma'am, is 
ma'am, is ````````````````````````````````````````````````````````````````````````````````````````````````````

Iteration #: 2
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 1.0126e-06
Generating from seed:  hearwhisp
 hearwhisp````````````````````````````````````````````````````````````````````````````````````````````````````

Iteration #: 3
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 7.9143e-07
Generating from seed:  but the h
 but the h````````````````````````````````````````````````````````````````````````````````````````````````````

Iteration #: 4
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 5.9605e-07
Generating from seed: nah!' 

#model 2